In [7]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
from itertools import chain
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import joblib

# Mis utilitarios
from utils import *


In [77]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Concatenate, ConvLSTM2D, Reshape
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import plot_model

# Set Path

In [9]:
el_path_main = Path(os.getcwd()).parent
el_path_main

WindowsPath('e:/backup Asus G15 27_10_2021/Colegio de Mates Bourbaki/DLA2501 - Deep Learning Avanzado/Flood forecasting')

In [10]:
# En caso sea en google Colab

# Load Tensors

In [11]:
X_gts_train, Y_train, X_gts_miss_train, X_ctry_lvl_train = joblib.load(el_path_main / '2. Datos/Processed data/TRAIN_data.pkl')
X_gts_oot, Y_oot, X_gts_miss_oot, X_ctry_lvl_oot = joblib.load(el_path_main / '2. Datos/Processed data/OOT_data.pkl')

In [12]:
# En caso sea en google Colab


In [13]:
X_gts_oot.shape, Y_oot.shape, X_gts_miss_oot.shape, X_ctry_lvl_oot.shape

((193, 10, 96, 36, 28), (193, 12, 36, 28), (193, 10, 96, 36, 28), (193, 180))

In [14]:
X_gts_train.shape, Y_train.shape, X_gts_miss_train.shape, X_ctry_lvl_train.shape

((1177, 10, 96, 36, 28),
 (1177, 12, 36, 28),
 (1177, 10, 96, 36, 28),
 (1177, 180))

In [112]:
def gen_X_for_model(X_gts, X_gts_miss):
    n_vars = X_gts.shape[1]
    lista_array = []
    for var in range(n_vars):
        X_tmp      = np.expand_dims(X_gts[:, var, :], -1)
        X_tmp_miss = np.expand_dims(X_gts_miss[:, var, :], -1)
        X = np.concat((X_tmp, X_tmp_miss), axis=-1)
        lista_array.append(X)
    return X

In [113]:
la_lista_de_array = gen_X_for_model(X_gts_oot, X_gts_miss_oot)

In [115]:
la_lista_de_array[0].shape

(96, 36, 28, 2)

# Conv-LSTM

In [95]:
n_vars_from_gts = X_gts_oot.shape[1] # N of variables from GTS
n_times_for_lstm = X_gts_oot.shape[2] # N of time steps usted for LSTM
n_lats = X_gts_oot.shape[3] # N of latitudes
n_lons = X_gts_oot.shape[4] # N of longitudes
vars_ctry_lvl = X_ctry_lvl_oot.shape[1] # N of variables from country level data
n_codmes_fcast = Y_oot.shape[1] # N of codmes for forecast

# Set of parameters that will be tunned
recurrent_dropout = 0.2
dropout = 0.2   
n_filters = 2
kernel_size = 3
include_miss_layer = True
dims_nn_ctry_lvl_tmp = [16] # La longitud de esta lista indica la cantidad de capas -1 (colcoar lista vacía para que sea de tamaño 1), los valores dentro indican el tamaño de la capa

# Set of parameters that are based on the previous ones
n_channels_conv_lstm = 2 if include_miss_layer else 1
dims_nn_ctry_lvl = dims_nn_ctry_lvl_tmp+[n_lats*n_lons*n_channels_conv_lstm] # La longitud de esta lista indica la cantidad de capas, los valores dentro indican el tamaño de la capa

In [104]:
# Input gts
dict_gts_inputs = {}
dict_gts_lstms  = {}
for i in range(n_vars_from_gts):

    # Inputs for each variable from GTS
    dict_gts_inputs[f'input_gts_{i}'] = Input(shape=(n_times_for_lstm, n_lats, n_lons, n_channels_conv_lstm, ), name=f'input_gts_{i}')

    # ConvLSTM2D for each variable from GTS
    dict_gts_lstms[f'lstm_gts_{i}'] = ConvLSTM2D(
                        filters = n_filters,
                        kernel_size = (kernel_size, kernel_size), # Tamaño del kernel
                        padding = 'same', # Para que la salida tenga el mismo tamaño que la entrada
                        data_format = 'channels_last', # Forma en que recibe los datos:Docu -> 5D tensor with shape: (samples, time, rows, cols, channels)
                        return_sequences = False, # No retorna cada secuencia de salida
                        # return_state=True, # Retorna los estados ocultos de la capa LSTM
                        recurrent_dropout = recurrent_dropout, dropout = dropout  # Para evitar sobreajuste
                        )(dict_gts_inputs[f'input_gts_{i}'])
    
# Input country level data
input_ctry_lvl = Input(shape=(vars_ctry_lvl,), name='input_ctry_lvl')
mi_dense_ctry_lvl = input_ctry_lvl

# Dense layers for country level data
for i in dims_nn_ctry_lvl:
    mi_dense_ctry_lvl = Dense(i, activation='relu')(mi_dense_ctry_lvl)

# Reshape so that it is compatible with the outputs of the LSTM.
mi_dense_ctry_lvl = Reshape((n_lats, n_lons, n_channels_conv_lstm))(mi_dense_ctry_lvl)

# Concatenate all the outputs of the LSTM layers and the dense layer for country level data
mi_concat = Concatenate()([dict_gts_lstms[f'lstm_gts_{i}'] for i in range(n_vars_from_gts)]+[mi_dense_ctry_lvl])

# Generate input of the decoder
input_decoder = Input(shape=(n_codmes_fcast, n_lats, n_lons, n_channels_conv_lstm), name='input_decoder')

# Generate Decoder ConvLSTM2D layer
decoder = ConvLSTM2D(filters=1,
                     kernel_size=(kernel_size, kernel_size),
                     padding='same',
                     return_sequences=True,
                     return_state=False)(input_decoder, initial_state=mi_concat)


mi_out = TimeDistributed(Dense(2, activation='softmax', name='mi_out'))(decoder)
mi_modelo = Model(inputs=list(dict_gts_inputs.values()), outputs=mi_out)
mi_modelo.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ctry_lvl      │ (None, 180)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_30 (Dense)    │ (None, 16)        │      2,896 │ input_ctry_lvl[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_0         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_1         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_2         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_3         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_4         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_5         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_6         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_7         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_8         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_gts_9         │ (None, 96, 36,    │          0 │ -                 │
│ (InputLayer)        │ 28, 2)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_31 (Dense)    │ (None, 2016)      │     34,272 │ dense_30[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_382     │ (None, 36, 28, 2) │        296 │ input_gts_0[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_383     │ (None, 36, 28, 2) │        296 │ input_gts_1[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_384     │ (None, 36, 28, 2) │        296 │ input_gts_2[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d_385     │ (None, 36, 28, 2) │        296 │ input_gts_3[0][0] │
│ (ConvLSTM2D)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 40,244 (157.20 KB)

 Trainable params: 40,244 (157.20 KB)

 Non-trainable params: 0 (0.00 B)

In [105]:
mi_modelo.compile(optimizer='adam',         # el optimizador sirve para encontrar los pesos que minimizan la función de pérdida
                                            # adam: stochastic gradient descent adaptativo
                                            # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
                  loss="categorical_crossentropy", # función que evalua que tan bien el algoritmo modela el conjunto de datos
                                                   # https://www.tensorflow.org/api_docs/python/tf/keras/losses
                  metrics=['accuracy'])

In [ ]:
history = mi_modelo.fit(x = [X_text_train],
                         y = y_train,
                        validation_split=0.2,
                        batch_size=128,
                        epochs=20,   # número de iteraciones sobre los datos
                        verbose=1)

In [103]:
decoder

<KerasTensor shape=(None, 12, 36, 28, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_499>

# Benchmark (Logit)